# Cryptocurrency prediction deep learning LSTM

### Predicción del precio de criptomonedas con deep learning

"Projecto" : "Análisis_criptomonedas"  
"Título" : "Predicción del precio de cryptomonedas con deep learning"  
"Autor" : "Cristian García Díaz"  
"Fecha de creación" : "20180821"  
"Fecha de modificación" : "20180826"  
"Fuentes":  
>https://medium.com/activewizards-machine-learning-company/bitcoin-price-forecasting-with-deep-learning-algorithms-eb578a2387a3

## Índice
[1. Configuración del entorno y obtención de los datos](#1)  
[2. Indicadores del mercado Bitcoin](#2)  
[3. Correlación de los indicadores de Bitcoin](#3)  
[4. Obtención y análisis del volumen cambio entre *Bitcoin* y divisas](#4) 

## <a name="1"></a> 1. Configuración del entorno y obtención de los datos

   - Instalar Anaconda.  
   - Instalar las librerias, dependencias y paquetes necesarios.  
   - Crear un entorno de trabajo.
   - Estructura de carpetas para almacenar los datos.  
   - Configuración de la API Key.  
   - Función para obtención de datos desde las APIs. 

In [1]:
# Se importan las líbrerias, dependencias o paquetes necesarios
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime
import plotly as py
import os

# Se importa el paquete Plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
# Se configura el modo offline
py.init_notebook_mode(connected=True)

from sklearn.preprocessing import MinMaxScaler

In [2]:
# API Key Quandl
quandl.ApiConfig.api_key = "gjodR_eNGkrTQq24cufg"

In [3]:
# Comprobar si no esta creada la carpeta de archivos para almacenar los datos
if not os.path.exists("cryptocurrency_indicators_files"):
    os.mkdir('cryptocurrency_indicators_files')

In [4]:
# Se define una función Quandl para cargar los datos
"""pickle --> para no descargar de nuevo los mismo datos"""
"""La función devuelve un Dataframe Pandas"""

def get_quandl_data(quandl_id):
    """Descargamos en cache los datos de Quandl"""
    """Se almacena un fichero .pkl como cache de los datos"""
    cache_path='.\cryptocurrency_indicators_files\{}.pkl'.format(quandl_id).replace('/','-')
    try:
        f = open(cache_path,'rb')
        df = pickle.load(f)
        print('Dataset {} cargado del cache'.format(quandl_id))
    except (OSError,IOError)as e:
        print('Descargando {} de Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas")
        df.to_pickle(cache_path)
        print('Cargado {} de {} en el cache'.format(quandl_id,cache_path))
    return df

In [5]:
# Se define la función para visualizar los datos
def df_scatter(df, title,seperate_y_axis=False, y_axis_label='',scale='linear',initial_hide=False):
    # Se definen la lista de los nombres de cada dataframe como una lista label_arr = ['BITSTAMP', 'COINBASE', 'ITBIT', 'KRAKEN']
    label_arr = list(df)
    # Aplicamos una función lambda para mapear cada columnas y asignar la etiqueta correspondiente
    # Se guarda como otra lista series_arr
    series_arr = list(map(lambda col:df[col],label_arr))
    
    # Se definen los parametros de la salida gráfica
    layout = go.Layout(
        title = title,
        legend = dict(orientation='h'),
        xaxis = dict(type='date'),
        yaxis = dict(
            title = y_axis_label, 
            showticklabels = not seperate_y_axis,
            type = scale
        )
    )
    
    # Se define la configuración del eje y
    y_axis_config = dict(
        overlaying = 'y',
        showticklabels = False,
        type = scale
    )
    
    # Se define la visibilidad
    visibility = 'visible'
    if initial_hide:
        visibility = 'legendonly'
        
    # Se define la forma para cada serie de datos
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
        x = series.index,
        y = series,
        name = label_arr[index],
        visible = visibility
        )
        
        #Añadir un eje separado para cada serie
        if seperate_y_axis:
            trace['yaxis'] = 'y{format}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config
        trace_arr.append(trace)
    
    fig = go.Figure(data = trace_arr, layout = layout)
    py.iplot(fig)

## <a name="2"></a> 2. Indicadores del mercado Bitcoin
   - [2.1 Precio de Bitcoin](#2.1)
   - [2.2 Número total de bictoins](#2.2)
   - [2.3 Valor del mercado](#2.3) 
   - [2.4 Direcciones de bitcoin](#2.4) 
   - [2.5 Volumen de cambio de bitcoin a dóladores](#2.5) 
   - [2.6 Número de transacciones de bitcoin](#2.6) 
   - [2.7 Número de transacciones acumuladas de bitcoin](#2.7) 
   - [2.8 Hash rate de bitcoin](#2.8) 
   - [2.9 Dificultad de bitcoin](#2.9) 
   - [2.10 Recompensa de los mineros de bitcoin](#2.10) 
   
   "Fuentes":  
>https://www.quandl.com/data/BCHAIN-Blockchain

### <a name="2.1"></a> 2.1 Precio de Bitcoin

In [6]:
# Obtención de los datos del precio
price_btc = get_quandl_data("BCHAIN/MKPRU")

Dataset BCHAIN/MKPRU cargado del cache


### <a name="2.2"></a> 2.2 Número total de bictoins

In [7]:
# Se obtienen los datos históricos del número total de Bitcoins
total_number_btc = get_quandl_data("BCHAIN/TOTBC")

Dataset BCHAIN/TOTBC cargado del cache


### <a name="2.3"></a> 2.3 Valor del mercado

In [8]:
# Se obtienen los datos históricos de la capitalización del mercado del Bitcoin en USD. El valor de mercado del Bitcoin
market_capitalization_btc = get_quandl_data("BCHAIN/MKTCP")

Dataset BCHAIN/MKTCP cargado del cache


### <a name="2.4"></a> 2.4 Número de direcciones bitcoin

In [9]:
# Se obtienen los datos históricos del número de direcciones Bitcoin usadas por dia
address_btc = get_quandl_data("BCHAIN/NADDU")

Dataset BCHAIN/NADDU cargado del cache


### <a name="2.5"></a> 2.5 Volumen de cambio de USD/BTC

In [10]:
# Se obtienen los datos históricos del volumen de cambio USD/BTC
exchange_trade_btc = get_quandl_data("BCHAIN/TRVOU")

Dataset BCHAIN/TRVOU cargado del cache


### <a name="2.6"></a> 2.6 Número de transacciones Bitcoin

In [11]:
# Se obtienen los datos históricos de las transacciones de BTC
transactions_btc = get_quandl_data("BCHAIN/NTRAN")

Dataset BCHAIN/NTRAN cargado del cache


### <a name="2.8"></a> 2.8 Hash rate de bitcoin 

In [12]:
# Se obtienen los datos históricos de Hash rate de bitcoin
# Es el número estimado de hash rate de Bitcoun y se miden en TeraHashes por segundo TH/s.
# 1 TH/s = 10^12 = 1.000.000.000.000 hash/s = 1 billón de hashes por segundo.
hash_rate_btc = get_quandl_data("BCHAIN/HRATE")

Dataset BCHAIN/HRATE cargado del cache


### <a name="2.9"></a> 2.9 Dificultad de bitcoin

In [13]:
# Se obtienen la dificultad de Bitcoin. Es una medida de dificultad propia.
# cada 210.000 bloques se recalcula la dificultad para crear bloques en la cadena de bloques cada 10 minutos de media.
difficulty_btc = get_quandl_data("BCHAIN/DIFF")

Dataset BCHAIN/DIFF cargado del cache


### <a name="2.10"></a> 2.10 Recompensa de los mineros de bitcoin 

In [14]:
# Se obtienen los datos históricos de la recompensa de los mineros
miners_revenue_btc = get_quandl_data("BCHAIN/MIREV")

Dataset BCHAIN/MIREV cargado del cache


## <a name="3"></a>3. Transformación de los datos

In [15]:
# Se preparan los datos para unirlos en un único Dataframe para poder aplicar la correlación.
mesures_name= ["price_btc",
        "total_number_btc",
        "market_capitalization_btc",
        "address_btc",
        "exchange_trade_btc",
        "transactions_btc",
        "hash_rate_btc",
        "difficulty_btc",
        "miners_revenue_btc"]

mesures_data= [price_btc,
        total_number_btc,
        market_capitalization_btc,
        address_btc,
        exchange_trade_btc,
        transactions_btc,
        hash_rate_btc,
        difficulty_btc,
        miners_revenue_btc]

In [16]:
# Preparación de un Dataframe con todos los indicadores e igualar los días para construir para todos los indicadores
fecha=pd.Timestamp(2018, 8, 26)

for i in range(len(mesures_name)):
    if(mesures_data[i].index.max()== fecha):
        mesures_data[i] = mesures_data[i].drop(mesures_data[i].index[len(mesures_data[i])-1])
    print( i, mesures_name[i], mesures_data[i].index.max()-mesures_data[i].index.min(), mesures_data[i].index.max())

0 price_btc 3521 days 00:00:00 2018-08-25 00:00:00
1 total_number_btc 3521 days 00:00:00 2018-08-25 00:00:00
2 market_capitalization_btc 3521 days 00:00:00 2018-08-25 00:00:00
3 address_btc 3521 days 00:00:00 2018-08-25 00:00:00
4 exchange_trade_btc 3521 days 00:00:00 2018-08-25 00:00:00
5 transactions_btc 3521 days 00:00:00 2018-08-25 00:00:00
6 hash_rate_btc 3520 days 00:00:00 2018-08-25 00:00:00
7 difficulty_btc 3521 days 00:00:00 2018-08-25 00:00:00
8 miners_revenue_btc 3521 days 00:00:00 2018-08-25 00:00:00


In [17]:
# Se realiza una unión de los varios Dataframe en uno
mesures_bitcoin = pd.concat(
    [mesures_data[0],
     mesures_data[1],
     mesures_data[2],
     mesures_data[3],
     mesures_data[4],
     mesures_data[5],
     mesures_data[6],
     mesures_data[7],
     mesures_data[8]],axis=1
)

In [18]:
# Renombrar las columnas
for i in range(len(mesures_name)):
    mesures_bitcoin.columns.values[i] = mesures_name[i]

In [19]:
# mesures_bitcoin = mesures_bitcoin.drop(['market_capitalization_btc'],axis=1)

In [20]:
# Crear los datos de entreno, test y evaluación
# Entreno
train_from_date = '2016-01-01'
train_end_date =  '2018-06-22'
# Test
test_from_date = '2018-06-23'
test_end_date = '2018-08-16'
# Evaluación
# '2018-08-17'
evaluation_from_date = '2018-08-10'
evaluation_end_date = '2018-08-22'

df_train = mesures_bitcoin.loc[train_from_date:train_end_date]
df_test = mesures_bitcoin.loc[test_from_date:test_end_date]
df_evaluation = mesures_bitcoin.loc[evaluation_from_date:evaluation_end_date]


print(df_train.size," días de entreno\n",df_test.size," días de test\n",df_evaluation.size," días de evaluación\n")

train_days = mesures_bitcoin.loc[train_from_date:train_end_date].count()
test_days = mesures_bitcoin.loc[test_from_date:test_end_date].count()
evalutacion_days = mesures_bitcoin.loc[evaluation_from_date:evaluation_end_date].count()
print(train_days, " desde ",train_from_date," hasta ",train_end_date )
print(test_days, " desde ",test_from_date," hasta ",test_end_date )
print(evalutacion_days, " desde ",evaluation_from_date," hasta ",evaluation_end_date )
print("Días totales ",train_days.price_btc+test_days.price_btc+evalutacion_days.price_btc)

8136  días de entreno
 495  días de test
 117  días de evaluación

price_btc                    904
total_number_btc             904
market_capitalization_btc    904
address_btc                  904
exchange_trade_btc           904
transactions_btc             904
hash_rate_btc                904
difficulty_btc               904
miners_revenue_btc           904
dtype: int64  desde  2016-01-01  hasta  2018-06-22
price_btc                    55
total_number_btc             55
market_capitalization_btc    55
address_btc                  55
exchange_trade_btc           55
transactions_btc             55
hash_rate_btc                55
difficulty_btc               55
miners_revenue_btc           55
dtype: int64  desde  2018-06-23  hasta  2018-08-16
price_btc                    13
total_number_btc             13
market_capitalization_btc    13
address_btc                  13
exchange_trade_btc           13
transactions_btc             13
hash_rate_btc                13
difficulty_btc        

# 4. Red neuronal LSTM


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from time import time
from math import sqrt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

import statsmodels.api as sm
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.callbacks import EarlyStopping
from keras import initializers

Using TensorFlow backend.


In [22]:
# Variable independientes
df_train_x = df_train.loc[:,"total_number_btc":]
# Variables predictoras
df_train_y = df_train.loc[:,"price_btc"]

# Variable independientes
df_test_x = df_test.loc[:,"total_number_btc":]
# Variables predictoras
df_test_y = df_test.loc[:,"price_btc"]

# Variable independientes
df_evaluation_x = df_evaluation.loc[:,"total_number_btc":]
# Variables predictoras
df_evaluation_y = df_evaluation.loc[:,"price_btc"]

In [23]:
# Se define la ventana para preparar los datos para LSTM
window_len = 10

In [24]:
# Se preparan los datos para unirlos en un único Dataframe para poder aplicar la correlación.
mesures_name= ["total_number_btc",
        "market_capitalization_btc",
        "address_btc",
        "exchange_trade_btc",
        "transactions_btc",
        "hash_rate_btc",
        "difficulty_btc",
        "miners_revenue_btc"]

In [25]:
# mesures_name.remove('market_capitalization_btc')

In [26]:
mesures_name

['total_number_btc',
 'market_capitalization_btc',
 'address_btc',
 'exchange_trade_btc',
 'transactions_btc',
 'hash_rate_btc',
 'difficulty_btc',
 'miners_revenue_btc']

NameError: name 'mesures_name' is not defined

In [27]:
# ENTRENO
#Variables explicativas
LSTM_training_inputs = []

for i in range(len(df_train_x)-window_len):
    temp_set = df_train_x[i:(i+window_len)].copy()
    for j in mesures_name:
        temp_set.loc[:, j] = temp_set[j]/temp_set[j].iloc[0] - 1
    LSTM_training_inputs.append(temp_set)
    
#Variable explicada
LSTM_training_outputs = (df_train_y[window_len:].values/df_train_y[:-window_len].values)-1

In [28]:
# TEST
#Variables explicativas
LSTM_test_inputs = []

for i in range(len(df_test_x)-window_len):
    temp_set = df_test_x[i:(i+window_len)].copy()
    for j in mesures_name:
        temp_set.loc[:, j] = temp_set[j]/temp_set[j].iloc[0] - 1
    LSTM_test_inputs.append(temp_set)
    
#Variable explicada 
LSTM_test_outputs = (df_test_y[window_len:].values/df_test_y[:-window_len].values)-1

In [29]:
# EVALUATION
#Variables explicativas
LSTM_evaluation_inputs = []

for i in range(len(df_evaluation_x)-window_len):
    temp_set = df_evaluation_x[i:(i+window_len)].copy()
    for j in mesures_name:
        temp_set.loc[:, j] = temp_set[j]/temp_set[j].iloc[0] - 1
    LSTM_evaluation_inputs.append(temp_set)
    
#Variable explicada 
LSTM_evaluation_outputs = (df_evaluation_y[window_len:].values/df_evaluation_y[:-window_len].values)-1

In [30]:
LSTM_training_inputs.count

<function list.count>

In [31]:
LSTM_training_outputs.size

894

In [32]:
LSTM_test_inputs.count

<function list.count>

In [33]:
LSTM_test_outputs.size

45

In [34]:
# Convertimos los datos de dataframe a matrices numpy ya que tenemos datos númericos
LSTM_training_inputs = [np.array(LSTM_training_input) for LSTM_training_input in LSTM_training_inputs]
LSTM_training_inputs = np.array(LSTM_training_inputs)

LSTM_test_inputs = [np.array(LSTM_test_inputs) for LSTM_test_inputs in LSTM_test_inputs]
LSTM_test_inputs = np.array(LSTM_test_inputs)

In [35]:
LSTM_evaluation_inputs = [np.array(LSTM_evaluation_inputs) for LSTM_evaluation_inputs in LSTM_evaluation_inputs]
LSTM_evaluation_inputs = np.array(LSTM_evaluation_inputs)

In [36]:
LSTM_evaluation_inputs.size

240

In [37]:
# LSTM_training_inputs.size =  894 lista de 10 listas de 7 listas = 62580 elementos
# 1 lista de 894 listas de 10 listas de 7 elementos
# 879 * 10 * 8
LSTM_training_inputs.size

71520

In [38]:
# LSTM_training_inputs.size =  45 lista de 10 listas de 7 listas = 3150 elementos
# 1 lista de 45 listas de 10 listas de 7 elementos
# 72 * 10 * 8
LSTM_test_inputs.size

3600

In [39]:
# Se importan los modulos necesarios
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [40]:
LSTM_training_outputs

array([ 3.93627428e-02,  3.41868480e-02,  2.93128082e-02,  3.36346437e-03,
       -2.54041570e-03, -9.32623292e-02, -1.40104469e-01, -1.50448524e-01,
       -1.38153185e-01, -1.58813729e-01, -8.49542847e-02, -8.74728814e-02,
       -1.36479649e-01, -9.56072351e-02, -6.28617736e-02, -1.15673357e-02,
        6.40388769e-02,  2.33493319e-02, -7.37126246e-03, -3.69724819e-03,
       -7.36903975e-02, -7.63235294e-02, -3.55198341e-02, -3.86615187e-02,
       -8.98579370e-02, -5.24425844e-03, -1.93595859e-02, -5.42781088e-02,
       -2.27486665e-02, -3.87001007e-03, -1.48847293e-02,  5.62543119e-03,
        1.27688172e-02,  2.13666060e-02,  5.48889733e-02,  4.49800031e-02,
        4.17863334e-02,  9.02048038e-02,  1.18531600e-01,  1.14874933e-01,
        1.21870052e-01,  1.59639031e-01,  1.53735899e-01,  1.49530750e-01,
        7.30828616e-02,  5.14936130e-02,  4.65924896e-02,  4.33991492e-02,
        3.33939336e-02,  3.46086832e-02,  4.34428975e-02, -1.73159188e-02,
       -1.07667886e-02, -

In [41]:
# La salida del modelo se normaliza
LSTM_training_outputs = (df_train['price_btc'][window_len:].values/df_train['price_btc'][:-window_len].values)-1

In [42]:
LSTM_training_outputs 

array([ 3.93627428e-02,  3.41868480e-02,  2.93128082e-02,  3.36346437e-03,
       -2.54041570e-03, -9.32623292e-02, -1.40104469e-01, -1.50448524e-01,
       -1.38153185e-01, -1.58813729e-01, -8.49542847e-02, -8.74728814e-02,
       -1.36479649e-01, -9.56072351e-02, -6.28617736e-02, -1.15673357e-02,
        6.40388769e-02,  2.33493319e-02, -7.37126246e-03, -3.69724819e-03,
       -7.36903975e-02, -7.63235294e-02, -3.55198341e-02, -3.86615187e-02,
       -8.98579370e-02, -5.24425844e-03, -1.93595859e-02, -5.42781088e-02,
       -2.27486665e-02, -3.87001007e-03, -1.48847293e-02,  5.62543119e-03,
        1.27688172e-02,  2.13666060e-02,  5.48889733e-02,  4.49800031e-02,
        4.17863334e-02,  9.02048038e-02,  1.18531600e-01,  1.14874933e-01,
        1.21870052e-01,  1.59639031e-01,  1.53735899e-01,  1.49530750e-01,
        7.30828616e-02,  5.14936130e-02,  4.65924896e-02,  4.33991492e-02,
        3.33939336e-02,  3.46086832e-02,  4.34428975e-02, -1.73159188e-02,
       -1.07667886e-02, -

# Entreno

In [43]:
# Se define una función para construir el modelo de red neuronal
# Se construye un modelo vacío sequencial y se agrega una capa LSTM.
# El modelo se ha configurado para que se adapte a una entrada n x m.
# Se incluye la función de activación.

def build_model(inputs, output_size, neurons, activ_func="linear",
                dropout=0.25, loss="mae", optimizer="adam"):
    model = Sequential()

    model.add(LSTM(neurons, input_shape=(inputs.shape[1], inputs.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model

In [44]:
# Se define una semilla para generar los números pseudoaleatorios
np.random.seed(202)

# Se inicializa el modelo
model_btc = build_model(LSTM_training_inputs, output_size=1, neurons = 20)

# La salida del modelo se normaliza
LSTM_training_outputs = (df_train['price_btc'][window_len:].values/df_train['price_btc'][:-window_len].values)-1

# Comprobar el tiempo
start_time = time()

#Se entrena al modelo. model_btc_history contiene información del error por entreno
model_btc_history = model_btc.fit(LSTM_training_inputs, LSTM_training_outputs, 
                            epochs=50, batch_size=1, verbose=2, shuffle=True)
# Comprobar el tiempo
final_time = time() - start_time

Epoch 1/50
 - 6s - loss: 0.0730
Epoch 2/50
 - 5s - loss: 0.0497
Epoch 3/50
 - 4s - loss: 0.0430
Epoch 4/50
 - 4s - loss: 0.0421
Epoch 5/50
 - 4s - loss: 0.0395
Epoch 6/50
 - 4s - loss: 0.0391
Epoch 7/50
 - 5s - loss: 0.0364
Epoch 8/50
 - 5s - loss: 0.0356
Epoch 9/50
 - 5s - loss: 0.0364
Epoch 10/50
 - 4s - loss: 0.0355
Epoch 11/50
 - 4s - loss: 0.0354
Epoch 12/50
 - 5s - loss: 0.0348
Epoch 13/50
 - 4s - loss: 0.0343
Epoch 14/50
 - 4s - loss: 0.0350
Epoch 15/50
 - 4s - loss: 0.0343
Epoch 16/50
 - 5s - loss: 0.0341
Epoch 17/50
 - 4s - loss: 0.0347
Epoch 18/50
 - 4s - loss: 0.0342
Epoch 19/50
 - 5s - loss: 0.0341
Epoch 20/50
 - 4s - loss: 0.0339
Epoch 21/50
 - 4s - loss: 0.0336
Epoch 22/50
 - 4s - loss: 0.0333
Epoch 23/50
 - 5s - loss: 0.0333
Epoch 24/50
 - 4s - loss: 0.0335
Epoch 25/50
 - 5s - loss: 0.0334
Epoch 26/50
 - 5s - loss: 0.0334
Epoch 27/50
 - 4s - loss: 0.0330
Epoch 28/50
 - 4s - loss: 0.0333
Epoch 29/50
 - 4s - loss: 0.0338
Epoch 30/50
 - 5s - loss: 0.0329
Epoch 31/50
 - 4s -

In [52]:
# Tiempo de ejecución
print('Tiempo de ejecución de la red neural es de: {0:.3f}'.format(final_time))

Tiempo de ejecución de la red neural es de: 223.351


In [53]:
# Gráfico del error MAE
history_error_btc = go.Scatter(x=model_btc_history.epoch, y=model_btc_history.history['loss'])
py.iplot([history_error_btc])

In [48]:
LSTM_training_inputs.shape

(894, 10, 8)

In [53]:
df_train_y.shape

(904,)

In [48]:
LSTM_training_inputs.shape

(894, 10, 8)

In [49]:
test =np.transpose(model_btc.predict(LSTM_training_inputs))+1

In [50]:
test.shape

(1, 894)

In [51]:
df_train_y.shape

(904,)

In [54]:
predicted = ((np.transpose(model_btc.predict(LSTM_training_inputs))+1) * df_train_y.values[:-window_len])[0]
observated = df_train_y.values[window_len:]

In [55]:
predicted.shape

(894,)

In [56]:
observated.shape

(894,)

In [57]:
observated[893]

6733.90166667

In [58]:
predicted[893]

6622.016321796942

In [59]:
# Visualización de 
trace1 = go.Scatter(
    x = np.arange(0, len(predicted), 1),
    y = predicted,
    mode = 'lines',
    name = 'Predicted',
    line = dict(color=('rgb(244, 146, 65)'), width=2)
)
trace2 = go.Scatter(
    x = np.arange(0, len(predicted), 1),
    y = observated,
    mode = 'lines',
    name = 'Observaciones',
    line = dict(color=('rgb(66, 244, 155)'), width=2)
)

data = [trace1, trace2]
layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our model predicted',
             xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='results_demonstrating0')

In [60]:
# MSE
print("MSE: %.3f" % mean_squared_error(observated, predicted))
# RMSE Root Mean Square Error
RMSE = sqrt(mean_squared_error(observated, predicted))
print('RMSE: %.3f' % RMSE)
from sklearn.metrics import mean_absolute_error
# MAE
print("MAE: %.3f" % mean_absolute_error(observated, predicted))

MSE: 102659.625
RMSE: 320.405
MAE: 135.575


In [118]:
# Ahora se extrae con el formato de fechas y se traza 
Test_Dates =df_train_x['total_number_btc'].index

trace1 = go.Scatter(x=Test_Dates, y=observated, name= 'Actual Price',
                   line = dict(color = ('rgb(66, 244, 155)'),width = 2))
trace2 = go.Scatter(x=Test_Dates, y=predicted, name= 'Predicted Price',
                   line = dict(color = ('rgb(244, 146, 65)'),width = 2))
data = [trace1, trace2]
layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our model predicted, by dates',
             xaxis = dict(title = 'Date'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='results_demonstrating1')

 ## Test

In [108]:
predicted_test = ((np.transpose(model_btc.predict(LSTM_test_inputs))+1) * df_test_y.values[:-window_len])[0]
observated_test = df_test_y.values[window_len:]

In [109]:
# Visualización de 
trace1 = go.Scatter(
    x = np.arange(0, len(predicted_test), 1),
    y = predicted_test,
    mode = 'lines',
    name = 'Predicted',
    line = dict(color=('rgb(244, 146, 65)'), width=2)
)
trace2 = go.Scatter(
    x = np.arange(0, len(observated_test), 1),
    y = observated_test,
    mode = 'lines',
    name = 'Observaciones',
    line = dict(color=('rgb(66, 244, 155)'), width=2)
)

data = [trace1, trace2]
layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our model predicted',
             xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='results_demonstrating0')

In [110]:
# MSE
print("MSE: %.3f" % mean_squared_error(observated_test, predicted_test))
# RMSE Root Mean Square Error
RMSE = sqrt(mean_squared_error(observated_test, predicted_test))
print('RMSE: %.3f' % RMSE)
from sklearn.metrics import mean_absolute_error
# MAE
print("MAE: %.3f" % mean_absolute_error(observated_test, predicted_test))


# TRAIN
# MSE: 102208.495
# RMSE: 319.701
# MAE: 134.233
# TEST
# MSE: 106684.829
# RMSE: 326.626
# MAE: 264.080

MSE: 107707.390
RMSE: 328.188
MAE: 262.578


In [167]:
# Ahora se extrae con el formato de fechas y se traza
# Se desplaza 10 dias por la ventana
# df_test.loc['2018-06-23':,'total_number_btc':'total_number_btc'].index

# Test_Dates = df_test['total_number_btc'].index

Test_Dates = df_test.loc['2018-07-03':,'total_number_btc':'total_number_btc'].index

trace1 = go.Scatter(x=Test_Dates, y=observated_test, name= 'Actual Price',
                   line = dict(color = ('rgb(66, 244, 155)'),width = 2))
trace2 = go.Scatter(x=Test_Dates, y=predicted_test, name= 'Predicted Price',
                   line = dict(color = ('rgb(244, 146, 65)'),width = 2))
data = [trace1, trace2]
layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our model predicted, by dates',
             xaxis = dict(title = 'Date'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='results_demonstrating1')

In [130]:
Test_Dates = df_test['total_number_btc'].index

In [129]:
T

array([False, False, False, False, False, False, False, False, False,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

## Evaluación

In [112]:
predicted_evaluation = ((np.transpose(model_btc.predict(LSTM_evaluation_inputs))+1) * df_evaluation_y.values[:-window_len])[0]
observated_evaluation = df_evaluation_y.values[window_len:]

In [113]:
# Visualización de 
trace1 = go.Scatter(
    x = np.arange(0, len(predicted_evaluation), 1),
    y = predicted_evaluation,
    mode = 'lines',
    name = 'Predicted',
    line = dict(color=('rgb(244, 146, 65)'), width=2)
)
trace2 = go.Scatter(
    x = np.arange(0, len(observated_evaluation), 1),
    y = observated_evaluation,
    mode = 'lines',
    name = 'Observaciones',
    line = dict(color=('rgb(66, 244, 155)'), width=2)
)

data = [trace1, trace2]
layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our model predicted',
             xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='results_demonstrating0')

In [114]:
# MSE
print("MSE: %.3f" % mean_squared_error(observated_evaluation, predicted_evaluation))
# RMSE Root Mean Square Error
RMSE = sqrt(mean_squared_error(observated_evaluation, predicted_evaluation))
print('RMSE: %.3f' % RMSE)
from sklearn.metrics import mean_absolute_error
# MAE
print("MAE: %.3f" % mean_absolute_error(observated_evaluation, predicted_evaluation))


# TRAIN
# MSE: 102208.495
# RMSE: 319.701
# MAE: 134.233
# TEST
# MSE: 106684.829
# RMSE: 326.626
# MAE: 264.080

MSE: 3110.589
RMSE: 55.773
MAE: 48.335


In [116]:
# Ahora se extrae con el formato de fechas y se traza 
Test_Dates =df_evaluation_x['total_number_btc'].index

trace1 = go.Scatter(x=Test_Dates, y=observated_evaluation, name= 'Actual Price',
                   line = dict(color = ('rgb(66, 244, 155)'),width = 2))
trace2 = go.Scatter(x=Test_Dates, y=predicted_evaluation, name= 'Predicted Price',
                   line = dict(color = ('rgb(244, 146, 65)'),width = 2))
data = [trace1, trace2]
layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our model predicted, by dates',
             xaxis = dict(title = 'Date'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='results_demonstrating1')